In [ ]:
import datetime
import os
import re

import dotenv
from pydantic import BaseModel, ConfigDict, Field
import requests

In [ ]:
dotenv.load_dotenv("../src/ynab-analysis/access.env", override=False)

In [ ]:
ynab_pat = os.environ["YNAB_PAT"]

In [ ]:
ynab_api_url_base = "https://api.ynab.com/v1"
budget_url = f"{ynab_api_url_base}/budgets"

In [ ]:
headers = {"Authorization": f"Bearer {ynab_pat}"}

In [ ]:
r = requests.get(budget_url, headers=headers)

In [ ]:
_class_to_underscore_separated("CurrencyFormat")

In [ ]:
def _class_to_underscore_separated(class_name: str) -> str:
    split = re.sub(
        "([A-Z][a-z]+)",
        r" \1",
        re.sub(
            "([A-Z]+)",
            r" \1",
            class_name
        )
    ).split()
    split = map(lambda x: x.lower(), split)
    return "_".join(split)



class CurrencyFormat(pydantic.BaseModel):
    model_config = ConfigDict(
        field_title_generator=lambda field_name, field_info: field_name,
        model_title_generator = lambda model: _class_to_underscore_separated(model.__name__)
    )
    
    iso_code: str = Field(default="USD", validate_default=True)
    decimal_digits: int = Field(default=2, validate_default=True)
    decimal_separator: str = Field(default=".", validate_default=True)
    group_separator: str = Field(default=",", validate_default=True)
    currency_separator: str = Field(default="$", validate_default=True)
    symbol_first: bool = Field(default=True, validate_default=True)
    display_symbol: bool = Field(default=True, validate_default=True)

class DateFormat(BaseModel):
    format_: str = Field(alias="format", default="MM/DD/YYYY")

class Budget(BaseModel):
    id: str = Field(frozen=True)
    name: str = Field()
    currency_format: CurrencyFormat = Field(default_factory=CurrencyFormat.__init__)
    last_modified_on: datetime.datetime
    first_month: datetime.datetime
    last_month: datetime.datetime
    date_format: DateFormat

class BudgetData(BaseModel):
    budgets: list[Budget]
    default_budget: str | None = None

In [ ]:
test_budget_data = r.json()["data"]["budgets"][0]

In [ ]:
ws_budget_data = BudgetData.model_validate(r.json()["data"])

In [ ]:
ws_budget = ws_budget_data.budgets[2]

In [ ]:
Budget.model_validate(test_budget_data)

In [ ]:
GET /budgets/{budget_id}/transactions

In [ ]:
transaction_template = "{ynab_api_url_base}/budgets/{budget_id}/transactions"

ws_transaction_request = transaction_template.format(ynab_api_url_base=ynab_api_url_base, budget_id=ws_budget.id)

In [ ]:
transaction_request = requests.get(ws_transaction_request, headers=headers)

In [ ]:
list(transaction_request.json()["data"]["transactions"][0].keys())

In [ ]:
transaction_request.json()["data"]["transactions"][0]

In [ ]:
for transaction in transaction_request.json()["data"]["transactions"]:
    if transaction["subtransactions"]:
        good_example = transaction.copy()
    else:
        good_example = None

In [ ]:
list(transaction_request.json()["data"].keys())

In [ ]:
transaction_request.json()["data"]["server_knowledge"]

In [ ]:
class Subtransaction(BaseModel):
    id_: str = Field(alias="id", frozen=True)
    transaction_id: str
    amount: float
    memo: str | None = Field(default=None)
    payee_id: str | None = Field(default=None)
    payee_name: str | None = Field(default=None)
    category_id: str | None = Field(default=None)
    category_name: str | None = Field(default=None)
    transfer_account_id: str | None = Field(default=None)
    transfer_transaction_id: str | None = Field(default=None)
    deleted: bool = Field(default=False)

In [ ]:
class Transaction(BaseModel):
    id_: str = Field(alias="id", frozen=True)
    date: datetime.datetime
    amount: int
    memo: str | None = Field(default=None)
    cleared: str | None = Field(default=None)  # should be an enum
    approved: bool
    account_id: str | None = Field(default=None)
    account_name: str | None = Field(default=None)
    payee_id: str | None = Field(default=None)
    payee_name: str | None = Field(default=None)
    transfer_transaction_id: str | None = Field(default=None)
    matched_transaction_id: str | None = Field(default=None)
    flag_color: str | None = Field(default=None)
    debt_transaction_type: str | None = Field(default=None)
    deleted: bool = Field(default=True)
    subtransactions: list[Subtransaction] = Field(default_factory=list)

In [ ]:
class TransactionData(BaseModel):
    transactions: list[Transaction]
    server_knowledge: int

In [ ]:
transaction_data = TransactionData.model_validate(transaction_request.json()["data"])